In [1]:
import json
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
# Import and store data
with open('One_Team.json', 'r') as file:
    data = json.load(file)

In [3]:
# Create a network
G = nx.DiGraph()

# Create nodes (patient pairs) for network
counter = 0
for element in data:
    G.add_nodes_from([(counter, element)])
    counter += 1

# Compatibility requirement
donor_recipient_compatibility = {'O': ['O', 'A', 'B', 'AB'], 'A': ['A', 'AB'], 'B':['B', 'AB'], 'AB':['AB']}

# Create edges (relations)
for node in G.nodes:
    donors = G.nodes[node]['Donor']
    for donor in donors:
        for vertex in G.nodes:
            if node == vertex: continue
            recipient = G.nodes[vertex]['Recipient']
            if recipient not in donor_recipient_compatibility[donor]: continue
            G.add_edge(node, vertex)    

In [4]:
# Find cycles of length 3

potential_cycle_3 = []

H = G.to_undirected()

for (u,v) in H.edges:
    for k in nx.common_neighbors(H, u, v):
        potential_cycle_3.append((u,v,k))
        
print("potential_cycle_3:", potential_cycle_3)        
        
cycle_3 = [] 

for (u,v,k) in potential_cycle_3:
    if (u,v) in G.edges and (v,k) in G.edges and (k,u) in G.edges: 
        cycle_3.append((u,v,k,u)) 
        
print("cycle_3:", cycle_3) 

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



cycle_3: []


In [5]:
# Find cycles of length 2

full_cycle_2 = []
short_cycle_2 = []

for (i,j) in G.edges:
    G.edges[(i,j)]["visited"] = False

for (u,v) in H.edges:
    if (u,v) in G.edges and G.edges[(u,v)]["visited"] == True: continue
    if (u,v) in G.edges and (v,u) in G.edges:
        full_cycle_2.append((u,v,u))
        short_cycle_2.append([u,v])
        G.edges[(u,v)]["visited"] = True

In [6]:
# Optimization
import gurobipy as gp
from gurobipy import GRB

# Create model object
m = gp.Model()

# Create variable for each edge
x = m.addVars(G.edges, vtype=GRB.BINARY)

# Objective function: maximize number of edges
m.setObjective(gp.quicksum(x[e] for e in G.edges), GRB.MAXIMIZE)

# The number of incomming arcs to each vertex is at most one
m.addConstrs(gp.quicksum(x[(u,v)] for u in G.neighbors(v) if (u,v) in G.edges) <= 1 for v in G.nodes)

# The number of incomming arcs should be equal to the number of outgoing arcs
m.addConstrs(gp.quicksum(x[(u,v)] for u in G.neighbors(v) if (u,v) in G.edges) == gp.quicksum(x[(v,u)] for u in G.neighbors(v) if (v,u) in G.edges) for v in G.nodes)

# If one edge is selected, the counterpart must be selected too
final_cycles = []
for (u,v) in short_cycle_2:
    if (u,v) in G.edges and (v,u) in G.edges:
        m.addConstr(x[(u,v)] == x[(v,u)])    
 
# Solve
m.optimize()

Set parameter Username
Set parameter LicenseID to value 2593705
Academic license - for non-commercial use only - expires 2025-12-01
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 34190 rows, 176760 columns and 373152 nonzeros
Model fingerprint: 0xe93a737e
Variable types: 0 continuous, 176760 integer (176760 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 33827 rows and 144028 columns
Presolve time: 0.25s
Presolved: 363 rows, 32732 columns, 65464 nonzeros
Variable types: 0 continuous, 32732 integer (32732 binary)
Found heuristic solution: objective 334.0000000

Starting sifting (using dua

In [7]:
# Exporting data for Excel to work with, and find some statistics since numpy is already imported
import numpy

selected_edges = [e for e in G.edges if x[e].X > 0.5]
selected_0 = []
selected_1 = []
averager = []


for tuple in selected_edges:
    selected_0.append(tuple[0])
    selected_1.append(tuple[1])
    difference = abs(tuple[0] - tuple[1])
    averager.append(difference)

numpy.transpose(selected_0)
numpy.transpose(selected_1)

average = numpy.average(averager)

print("Average donors needed to find a match in this population: ", average)
print("Unmatched recipients:", (len(G.nodes) - len(selected_edges)), "out of", len(G.nodes))

with open('Approved_Surgeries_0.json', 'w') as file:
    json.dump(selected_0, file)

with open('Approved_Surgeries_1.json', 'w') as file:
    json.dump(selected_1, file)

with open('One_Team_Approved_Surgeries.json', 'w') as file:
    json.dump(final_cycles, file)

with open('cycless.json', 'w') as file:
    json.dump(short_cycle_2, file)

Average donors needed to find a match in this population:  50.461077844311376
Unmatched recipients: 395 out of 729
